In [6]:
import numpy as np # linear algebra
import pandas as pd # data processing
import os
from keras.layers import Dense,Input,LSTM,Bidirectional,Activation,Conv1D,GRU
from keras.callbacks import Callback
from keras.layers import Dropout,Embedding,GlobalMaxPooling1D, MaxPooling1D, Add, Flatten
from keras.preprocessing import text, sequence
from keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate, SpatialDropout1D
from keras import initializers, regularizers, constraints, optimizers, layers, callbacks
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.models import Model
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn import metrics

EMBEDDING_FILE = '/Users/isabelmetzger/PycharmProjects/BMSC-GA-4493-Spring2018/lab_0322/glove.840B.300d.txt'
train = pd.read_csv("/Users/isabelmetzger/PycharmProjects/n2c2/LSTM_BASELINES/data/train_May4th_binary.csv")

y_train = train[["ABDOMINAL","ADVANCED_CAD","ALCOHOL_ABUSE",
               "ASP_FOR_MI","CREATININE","DIETSUPP_2MOS",
               "DRUG_ABUSE","ENGLISH","HBA1C","KETO_1YR",
               "MAJOR_DIABETES", "MAKES_DECISIONS", "MI_6MOS"]].values

val = pd.read_csv("/Users/isabelmetzger/PycharmProjects/n2c2/LSTM_BASELINES/data/test_May4th_binary.csv")
y_val = val[["ABDOMINAL","ADVANCED_CAD","ALCOHOL_ABUSE",
               "ASP_FOR_MI","CREATININE","DIETSUPP_2MOS",
               "DRUG_ABUSE","ENGLISH","HBA1C","KETO_1YR",
               "MAJOR_DIABETES", "MAKES_DECISIONS", "MI_6MOS"]].values
test = pd.read_csv("/Users/isabelmetzger/Downloads/validation.csv")

print("datasets uploaded!")

import re
def clean_text(text):
    text = text.lower()
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub('\W', ' ', text)
    text = re.sub('\s+', ' ', text)
    text = text.strip(' ')
    return text

X_train = train["text"].map(lambda com : clean_text(com))
# y_train = train[['ABDOMINAL', 'ADVANCED.CAD', 'CREATININE', 'HBA1C',
#                  'MAJOR.DIABETES', 'MI.6MOS']].values

X_val = val["text"].map(lambda com : clean_text(com))
X_test = test["textdata"].map(lambda com : clean_text(com))
# y_test = test[['ABDOMINAL', 'ADVANCED.CAD', 'CREATININE', 'HBA1C',
#                  'MAJOR.DIABETES', 'MI.6MOS']].values

max_features=28160
maxlen=21500
embed_size=300
#21583

class RocAucEvaluation(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()

        self.interval = interval
        self.X_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.X_val, verbose=0)
            score = roc_auc_score(self.y_val, y_pred)
            f1_score = metrics.f1_score(self.y_val, y_pred)
            print("\n ROC-AUC - epoch: {:d} - roc acc score: {:.6f}".format(epoch+1, score))
            print("\n F1 score - epoch: {:d} - F1 score: {:.6f}".format(epoch+1, f1_score))
            
tok=text.Tokenizer(num_words=max_features,lower=False)
tok.fit_on_texts(list(X_train)+ list(X_val) + list(X_test))
X_train=tok.texts_to_sequences(X_train)
X_val=tok.texts_to_sequences(X_val)
X_test=tok.texts_to_sequences(X_test)
x_train=sequence.pad_sequences(X_train,maxlen=maxlen)
x_val=sequence.pad_sequences(X_val, maxlen=maxlen)
x_test=sequence.pad_sequences(X_test,maxlen=maxlen)

datasets uploaded!


In [7]:
embeddings_index = {}
with open(EMBEDDING_FILE,encoding='utf8') as f:
    for line in f:
        values = line.rstrip().rsplit(' ')
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
        
print("embedding file downloaded!")

embedding file downloaded!


In [ ]:
len(tok.word_index)

In [ ]:
 # 21582
word_index = tok.word_index
print(len(word_index))
num_words = min(max_features, len(word_index) + 1)
print(num_words)  # 10000

In [7]:
word_index = tok.word_index
#prepare embedding matrix
num_words = min(max_features, len(word_index) + 1)
embedding_matrix = np.zeros((num_words, embed_size))
for word, i in word_index.items():
    if i >= max_features:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

print(embedding_matrix.size)  # 8448000


embedding_matrix.shape  # (28160, 300)

In [10]:
from keras.layers import K, Activation
from keras.engine import Layer
from keras.layers import Dense, Input, Embedding, Dropout, Bidirectional, GRU, Flatten, SpatialDropout1D
gru_len = 128
Routings = 5
Num_capsule = 10
Dim_capsule = 16
dropout_p = 0.25
rate_drop_dense = 0.5

def squash(x, axis=-1):
    # s_squared_norm is really small
    # s_squared_norm = K.sum(K.square(x), axis, keepdims=True) + K.epsilon()
    # scale = K.sqrt(s_squared_norm)/ (0.5 + s_squared_norm)
    # return scale * x
    s_squared_norm = K.sum(K.square(x), axis, keepdims=True)
    scale = K.sqrt(s_squared_norm + K.epsilon())
    return x / scale

import keras.backend as K

# def f1_score(y_true, y_pred):
#     return %precision

# A Capsule Implement with Pure Keras
class Capsule(Layer):
    def __init__(self, num_capsule, dim_capsule, routings=3, kernel_size=(9, 1), share_weights=True,
                 activation='default', **kwargs):
        super(Capsule, self).__init__(**kwargs)
        self.num_capsule = num_capsule
        self.dim_capsule = dim_capsule
        self.routings = routings
        self.kernel_size = kernel_size
        self.share_weights = share_weights
        if activation == 'default':
            self.activation = squash
        else:
            self.activation = Activation(activation)

    def build(self, input_shape):
        super(Capsule, self).build(input_shape)
        input_dim_capsule = input_shape[-1]
        if self.share_weights:
            self.W = self.add_weight(name='capsule_kernel',
                                     shape=(1, input_dim_capsule,
                                            self.num_capsule * self.dim_capsule),
                                     # shape=self.kernel_size,
                                     initializer='glorot_uniform',
                                     trainable=True)
        else:
            input_num_capsule = input_shape[-2]
            self.W = self.add_weight(name='capsule_kernel',
                                     shape=(input_num_capsule,
                                            input_dim_capsule,
                                            self.num_capsule * self.dim_capsule),
                                     initializer='glorot_uniform',
                                     trainable=True)

    def call(self, u_vecs):
        if self.share_weights:
            u_hat_vecs = K.conv1d(u_vecs, self.W)
        else:
            u_hat_vecs = K.local_conv1d(u_vecs, self.W, [1], [1])

        batch_size = K.shape(u_vecs)[0]
        input_num_capsule = K.shape(u_vecs)[1]
        u_hat_vecs = K.reshape(u_hat_vecs, (batch_size, input_num_capsule,
                                            self.num_capsule, self.dim_capsule))
        u_hat_vecs = K.permute_dimensions(u_hat_vecs, (0, 2, 1, 3))
        # final u_hat_vecs.shape = [None, num_capsule, input_num_capsule, dim_capsule]

        b = K.zeros_like(u_hat_vecs[:, :, :, 0])  # shape = [None, num_capsule, input_num_capsule]
        for i in range(self.routings):
            b = K.permute_dimensions(b, (0, 2, 1))  # shape = [None, input_num_capsule, num_capsule]
            c = K.softmax(b)
            c = K.permute_dimensions(c, (0, 2, 1))
            b = K.permute_dimensions(b, (0, 2, 1))
            outputs = self.activation(K.batch_dot(c, u_hat_vecs, [2, 2]))
            if i < self.routings - 1:
                b = K.batch_dot(outputs, u_hat_vecs, [2, 3])

        return outputs

    def compute_output_shape(self, input_shape):
        return (None, self.num_capsule, self.dim_capsule)


def get_model():
    input1 = Input(shape=(maxlen,))
    embed_layer = Embedding(max_features,
                            embed_size,
                            input_length=maxlen,
                            weights=[embedding_matrix],
                            trainable=False)(input1)
    embed_layer = SpatialDropout1D(rate_drop_dense)(embed_layer)

    x = Bidirectional(
        GRU(gru_len, activation='relu', dropout=dropout_p, recurrent_dropout=dropout_p, return_sequences=True))(
        embed_layer)
    capsule = Capsule(num_capsule=Num_capsule, dim_capsule=Dim_capsule, routings=Routings,
                      share_weights=True)(x)
    # output_capsule = Lambda(lambda x: K.sqrt(K.sum(K.square(x), 2)))(capsule)
    capsule = Flatten()(capsule)
    capsule = Dropout(dropout_p)(capsule)
    output = Dense(6, activation='sigmoid')(capsule)
    model = Model(inputs=input1, outputs=output)
    model.compile(
        loss='binary_crossentropy',
        optimizer='adam',
        metrics=['binary_accuracy'])
    model.summary()
    return model

model = get_model()

In [12]:
#X_tra, X_val, y_tra, y_val = train_test_split(x_train, y_train, train_size=0.85, random_state=233)
#RocAuc = RocAucEvaluation(validation_data=(X_val, y_val), interval=1)
#RocAuc

/Users/isabelmetzger/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [13]:
batch_size = 256
epochs = 4

hist = model.fit(X_tra, y_tra, batch_size=batch_size, epochs=1, validation_data=(X_val, y_val), verbose=1)

Train on 137 samples, validate on 25 samples
Epoch 1/1
137/137 [==============================] - 3s 22ms/step - loss: 0.7404 - binary_accuracy: 0.4331 - val_loss: 0.6382 - val_binary_accuracy: 0.6533


In [14]:
model.fit(X_tra, y_tra, batch_size=batch_size, epochs=epochs, validation_data=(X_val, y_val), verbose=1)

y_pred = model.predict(x_test, batch_size=1024, verbose=1)


Train on 137 samples, validate on 25 samples
Epoch 1/4
137/137 [==============================] - 1s 8ms/step - loss: 0.6545 - binary_accuracy: 0.6010 - val_loss: 0.6102 - val_binary_accuracy: 0.6600
Epoch 2/4
137/137 [==============================] - 1s 9ms/step - loss: 0.6202 - binary_accuracy: 0.6557 - val_loss: 0.6053 - val_binary_accuracy: 0.6600
Epoch 3/4
137/137 [==============================] - 1s 8ms/step - loss: 0.6165 - binary_accuracy: 0.6533 - val_loss: 0.6042 - val_binary_accuracy: 0.6600
Epoch 4/4
40/40 [==============================] - 0s 10ms/step


In [15]:
capsule_net_y_pred_DF = pd.DataFrame(y_pred)
#y_pred.to_csv("y_predicted_capsule_net.csv", )
capsule_net_y_pred_DF.columns = ['ABDOMINAL', 'ADVANCED.CAD', 'CREATININE', 'HBA1C', 'MAJOR.DIABETES', 'MI.6MOS']
capsule_net_y_pred_DF.head()
#.to_csv("capsue_net_y_pred_DF_April_10_2018.csv")

,ABDOMINAL,ADVANCED.CAD,CREATININE,HBA1C,MAJOR.DIABETES,MI.6MOS
0,0.343666,0.627940,0.403582,0.237049,0.662170,0.085365
1,0.334830,0.641155,0.408596,0.235950,0.659623,0.085893
2,0.332630,0.635828,0.411987,0.235938,0.661332,0.080346
3,0.337645,0.606472,0.409643,0.235415,0.654351,0.085237
4,0.323511,0.646254,0.412110,0.249115,0.649626,0.077680


In [16]:
capsule_net_y_pred_DF.to_csv('capsule_net_y_pred_DF_April_10_2018.csv')